# Ames Housing Data and Kaggle Challenge
## KAGGLE SUBMISSION
***

*Import libraries and read in data:*

In [1]:
# Imports

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score

%matplotlib inline

In [2]:
# Read in the train and test data files and replaced all NaNs with 'NA'
train = pd.read_csv('../datasets/train_clean_dummies.csv', na_filter = False,)
test  = pd.read_csv('../datasets/test_clean_dummies.csv', na_filter = False)

# drop 'Unnamed: 0' column that was created when we saved the cleaned dataset (old index)
train.drop(columns = 'Unnamed: 0', inplace = True)
test.drop(columns = 'Unnamed: 0', inplace = True)

### Power Transformation and Model Selection
***

In [3]:
# Select features to use
top_features = abs(train.corr()['SalePrice']).sort_values(ascending=False).keys()
features = [col for col in top_features if col != 'SalePrice']

In [4]:
# Create X and y variables
X = train[features]
y = train['SalePrice']

In [5]:
# Power Transformer
pt = PowerTransformer()
pt.fit(X)
X_pt = pt.transform(X)
X_test_pt = pt.transform(test[features])



pt_y = PowerTransformer()
# PowerTransformer requires a matrix/DataFrame, which is why we're using the .to_frame() method on y_train
pt_y.fit(y.to_frame()) 
y_pt = pt_y.transform(y.to_frame())


/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:121: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:2798: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [6]:
# Instantiate and score models
lr = LinearRegression()
lasso = LassoCV(n_alphas=200, cv=5)
ridge = RidgeCV(alphas=np.linspace(.1, 10, 100))

# Cross Validation
print('LR Score: ', cross_val_score(lr, X_pt, y_pt, cv=3).mean())
print('Lasso Score: ', cross_val_score(lasso, X_pt, y_pt[:, 0], cv=5).mean())
print('Ridge Score: ', cross_val_score(ridge, X_pt, y_pt, cv=3).mean())

LR Score:  0.918056333800291
Lasso Score:  0.9201369558439506
Ridge Score:  0.9184563852137994


In [7]:
# Fit best scoring model
lasso.fit(X_pt, y_pt)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=200, n_jobs=None, normalize=False,
    positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=False)

In [8]:
# Score chosen model
lasso.score(X_pt, y_pt)

0.9265580060426809

In [9]:
# # Get r2 score
# pred = lasso.predict(X_pt)
# # The .reshape(-1,1) method changes a numpy array into a numpy matrix with 1 column
# pred_reversed = pt_y.inverse_transform(pred.reshape(-1,1))
# r2_score(y_, pred_reversed)

### Transform Test & Make Prediction
***

In [10]:
# Power transform with train dataset to make predictions
pred = lasso.predict(X_test_pt)
pred_reversed = pt_y.inverse_transform(pred.reshape(-1,1))

In [11]:
# Create submission DataFrame
submission = test[['Id']]
submission['SalePrice'] = pred_reversed
submission.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Id,SalePrice
0,2658,135405.799758
1,2718,155896.397331
2,2414,221555.020765
3,1989,100430.522770
4,625,169221.983472


In [12]:
# Save submission to CSV
submission.to_csv('submission4.csv', index = False)